https://github.com/langchain-ai/open_deep_research

In [1]:
# ! pip install -U -q open-deep-research

In [ ]:
import open_deep_research   
print(open_deep_research.__version__) 

import os

os.environ["AZURE_OPENAI_API_KEY"] = "XXX"
os.environ["AZURE_OPENAI_API_VERSION"] = "XXX"
os.environ["OPENAI_API_VERSION"] = "XXX"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "XXX"
os.environ["MODEL_PROVIDER"] = "azure_openai"
os.environ["AZURE_OPENAI_ENDPOINT"] = "XXX"

os.environ["TAVILY_API_KEY"] = "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"


from IPython.display import Image, display
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from open_deep_research.graph import builder

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
# display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

# import os, getpass

# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# # Set the API keys used for any model or search tool selections below, such as:
# _set_env("OPENAI_API_KEY")
# _set_env("TAVILY_API_KEY")
# _set_env("OPENAI_API_VERSION")

0.0.10


In [7]:
import uuid 
from IPython.display import Markdown

REPORT_STRUCTURE = """Use this structure to create a report on the user-provided topic:

1. Introduction (no research needed)
   - Brief overview of the topic area

2. Main Body Sections:
   - Each section should focus on a sub-topic of the user-provided topic
   
3. Conclusion
   - Aim for 1 structural element (either a list of table) that distills the main body sections 
   - Provide a concise summary of the report"""

thread = {"configurable": {"thread_id": str(uuid.uuid4()),
                           "search_api": "tavily",
                           "planner_provider": os.environ["MODEL_PROVIDER"],
                           "planner_model": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
                           "writer_provider": os.environ["MODEL_PROVIDER"],
                           "writer_model": os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"], 
                           "report_structure": REPORT_STRUCTURE,
                           "max_search_depth": 1,}
                           } 
# Create a topic
topic = "Overview of the AI inference market with focus on Fireworks, Together.ai, Groq"

# Run the graph until the interruption
async for event in graph.astream({"topic":topic,}, thread, stream_mode="updates"):
    if '__interrupt__' in event:
        interrupt_value = event['compile_final_report']['final_report']
        display(Markdown(interrupt_value))

yes right


In [ ]:
# # Pass feedback to update the report plan  
# async for event in graph.astream(Command(resume="Include individuals sections for Together.ai, Groq, and Fireworks with revenue estimates (ARR)"), thread, stream_mode="updates"):
#     if '__interrupt__' in event:
#         interrupt_value = event['__interrupt__'][0].value
#         display(Markdown(interrupt_value))